In [1]:
# 필요한 모듈 설치
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu

     -------------------------------------- 227.6/227.6 kB 3.4 MB/s eta 0:00:00
     --------------------------------------- 14.0/14.0 MB 19.8 MB/s eta 0:00:00
     ---------------------------------------- 153.2/153.2 kB ? eta 0:00:00
     ------------------------------------- 162.6/162.6 MB 21.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple/
     ------------------------------------- 136.5/136.5 MB 22.6 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 25.5 MB/s eta 0:00:00
     ---------------------------------------- 268.8/268.8 kB ? eta 0:00:00
     ---------------------------------------- 78.4/78.4 kB 4.3 MB/s eta 0:00:00
     ------------------------------------- 269.6/269.6 kB 16.2 MB/s eta 0:00:00
     ---------------------------------------- 62.6/62.6 kB ? eta 0:00:00
     ---------------------------------------- 286.2/286.2 kB ? eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 8.5 MB/s eta 0:00:00
     ---------------

ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=3.8.1,<4.0; 0.0.10 Requires-Python >=3.8.1,<4.0; 0.0.11 Requires-Python >=3.8.1,<4.0; 0.0.12 Requires-Python >=3.8.1,<4.0; 0.0.13 Requires-Python >=3.8.1,<4.0; 0.0.14 Requires-Python >=3.8.1,<4.0; 0.0.15 Requires-Python >=3.8.1,<4.0; 0.0.16 Requires-Python >=3.8.1,<4.0; 0.0.17 Requires-Python >=3.8.1,<4.0; 0.0.18 Requires-Python >=3.8.1,<4.0; 0.0.19 Requires-Python >=3.8.1,<4.0; 0.0.1rc1 Requires-Python >=3.8.1,<4.0; 0.0.1rc2 Requires-Python >=3.8.1,<4.0; 0.0.2 Requires-Python >=3.8.1,<4.0; 0.0.20 Requires-Python >=3.8.1,<4.0; 0.0.21 Requires-Python >=3.8.1,<4.0; 0.0.22 Requires-Python >=3.8.1,<4.0; 0.0.23 Requires-Python >=3.8.1,<4.0; 0.0.24 Requires-Python >=3.8.1,<4.0; 0.0.25 Requires-Python >=3.8.1,<4.0; 0.0.26 Requires-Python >=3.8.1,<4.0; 0.0.27 Requires-Python >=3.8.1,<4.0; 0.0.28 Requires-Python >=3.8.1,<4.0; 0.0.29 Requires-Python <4.0,>=3.8.1; 0.0.3 Requires-Python >=3.8.1,<

     --------------------------------------- 11.7/11.7 MB 40.9 MB/s eta 0:00:00
     ---------------------------------------- 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.1/1.1 MB 23.5 MB/s eta 0:00:00
     ---------------------------------------- 7.1/7.1 MB 50.7 MB/s eta 0:00:00
     --------------------------------------- 34.1/34.1 MB 40.9 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 48.4 MB/s eta 0:00:00
     ------------------------------------- 991.7/991.7 kB 61.3 MB/s eta 0:00:00
     ------------------------------------- 302.2/302.2 kB 19.5 MB/s eta 0:00:00
     ---------------------------------------- 97.9/97.9 kB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 53.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=

In [7]:
!pip install torch
!pip install transformers
!pip install pydantic==1.10.9

     ---------------------------------------- 2.1/2.1 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.3
    Uninstalling pydantic-2.5.3:
      Successfully uninstalled pydantic-2.5.3


In [14]:
# 필요한 모듈 import
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# from peft import PeftModel

PydanticUserError: If you use `@root_validator` with pre=False (the default) you MUST specify `skip_on_failure=True`. Note that `@root_validator` is deprecated and should be replaced with `@model_validator`.

For further information visit https://errors.pydantic.dev/2.5/u/root-validator-pre-skip

## 우리가 코드에 넣을 수 있는 개선사항
1. 이건 Llama3로 학습시킨 모델이 사용한 방법

    a. 라마3로 학습해서 공유한 코드에는 pypdf 대신 pymupdf를 사용해서 추출 정확도를 높임
    
    b. retriever를 가중치를 적용하여 kiwi 한국어 분석기를 사용

2. 우리는 모델을 yanolja 버전으로 사용해서 좀 더 한국어를 잘 이해하는 모델을 사용할 수 있음

3. Prompt를 langshmith에서 불러와서 rag에 맞는 프롬프트를 사용

4.  임베딩 과정에서 LLM이 받을 수 있는 적당한 양의 토큰 수로 조절

In [ ]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    index = faiss.IndexFlatL2(1536)
    # FAISS DB 생성 및 반환
    from langchain.docstore import InMemoryDocstore
    db = FAISS(embeddings, index, InMemoryDocstore({}), {})
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)

        # Retriever 생성
        ## 시간에 따라 가중치가 붙는 retriever를 적용시킴 최근에 접근한 적이 있으면 더 우선적으로 정보를 선택
        from langchain.retrievers import TimeWeightedVectorStoreRetriever
        retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=db, decay_rate=0.0000000000000000000000001, k=1
)

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


In [ ]:
base_directory = './open' # Your Base Directory
df = pd.read_csv('./open/test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Processing 중소벤처기업부_혁신창업사업화자금(융자)...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:53<00:13,  6.81s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [01:00<00:06,  6.76s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [01:05<00:00,  7.26s/it]


In [ ]:
# 모델과 토크나이저 호출
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID
    ## 야놀자 llm모델로 id 변경 (일단 가벼운 모델로 테스트)
    model_id = "kyujinpy/Ko-PlatYi-6B"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,  # 양자화 적용 부분
        device_map="auto",
        trust_remote_code=True )

    model = PeftModel.from_pretrained(model, "./persona/checkpoint-200",is_trainable=True)

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2, # 좀 더 일관성 있는 대답으로
        return_full_text=False,
        max_new_tokens=300,  # 토큰수 조절
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

In [ ]:
llm = setup_llm_pipeline()

tokenizer_config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성

    ## prompt를 langsmith에서 한국어로 적용된 프롬프트를 불러옴
    from langchain import hub
    prompt = hub.pull("teddynote/rag-prompt-korean")

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)